# PROBLEM SET 2

In [1]:
import pandas as pd

df = pd.read_csv('/Users/jugalmarfatia/Documents/fall 2018/econometrics_2/matching_estimators/bwght2.csv')

## Replace blank cells (NaN) with 0. 

In [2]:
df = df.where((pd.notnull(df)), 0)

# Display head to check if replaced 
df.head()

,mage,meduc,monpre,npvis,fage,feduc,bwght,omaps,fmaps,cigs,...,male,mwhte,mblck,moth,fwhte,fblck,foth,lbwght,magesq,npvissq
0,26,12.0,2.0,12.0,34.0,16.0,3060,9.0,9.0,0.0,...,1,0,0,1,0,0,1,8.026170,676,144.0
1,29,12.0,2.0,12.0,32.0,12.0,3730,8.0,9.0,0.0,...,0,1,0,0,1,0,0,8.224163,841,144.0
2,33,12.0,1.0,12.0,36.0,16.0,2530,8.0,9.0,0.0,...,0,1,0,0,1,0,0,7.835975,1089,144.0
3,28,17.0,5.0,8.0,32.0,17.0,3289,8.0,9.0,0.0,...,1,1,0,0,1,0,0,8.098339,784,64.0
4,23,13.0,2.0,6.0,24.0,16.0,3590,6.0,8.0,0.0,...,1,1,0,0,1,0,0,8.185907,529,36.0


## Create dummy variable for ever smoked

In [3]:
df['cigs_dummy'] = (df['cigs'] > 0).astype(int)

## Difference in log birthweight of smokers vs non-smokers

In [4]:
s_lbwght, ns_lbwght = df.groupby('cigs_dummy')['lbwght'].mean()
diff = (s_lbwght - ns_lbwght)

In [5]:
print "Difference in log birthweight = " + str(diff)

Difference in log birthweight = 0.0595264954765


## Run OLS regression 

In [6]:
import statsmodels.formula.api as sm
result = sm.ols(formula="lbwght ~ cigs_dummy + mage + meduc + monpre + npvis + fage + feduc + fblck + magesq + npvissq + mblck", 
                data=df).fit()



## Results from the regression. 

In [7]:
print result.summary()


                            OLS Regression Results                            
Dep. Variable:                 lbwght   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     4.702
Date:                Tue, 04 Sep 2018   Prob (F-statistic):           3.92e-07
Time:                        14:48:47   Log-Likelihood:                 346.30
No. Observations:                1832   AIC:                            -668.6
Df Residuals:                    1820   BIC:                            -602.4
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      7.5400      0.140     54.037      0.0